In [7]:
from mintalib import core
from mintalib import functions as fx
from mintalib.samples import sample_prices


class PricesMethods:
    _prices = None

    def dispatch_prices(self, func, *args, **kwds):
        result = func(self._prices, *args, **kwds)
        return core.wrap_result(result, self._prices)

    def atr(self, period: int = 14):
        kwds = dict(period=period)
        return self.dispatch_prices(core.calc_atr, **kwds)



class SeriesMethods:
    _series = None

    def dispatch_series(self, func, *args, **kwds):
        result = func(self._series, *args, **kwds)
        return core.wrap_result(result, self._series)

    def ema(self, period: int = 20):
        kwds = dict(period=period)
        return self.dispatch_series(core.calc_ema, **kwds)

    def sma(self, period: int = 20):
        kwds = dict(period=period)
        return self.dispatch_series(core.calc_sma, **kwds)


def register_pandas(name="ta"):
    import pandas as pd

    @pd.api.extensions.register_dataframe_accessor(name)
    class PricesAccessor(PricesMethods):
        def __init__(self, prices):
            self._validate(prices)
            self._prices = prices

        @staticmethod
        def _validate(prices):
            if "close" not in prices.columns:
                raise AttributeError("Prices must have a 'close' column") 

        @property
        def _series(self):
            return self._prices


    @pd.api.extensions.register_series_accessor(name)
    class SeriesAccessor(SeriesMethods):
        def __init__(self, series):
            self._series = series


register_pandas()


/var/folders/s8/jtb72jcx6739t9rmc8pl26980000gn/T/ipykernel_37820/1702341758.py:39: UserWarning: registration of accessor <class '__main__.register_pandas.<locals>.PricesAccessor'> under name 'ta' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class PricesAccessor(PricesMethods):
/var/folders/s8/jtb72jcx6739t9rmc8pl26980000gn/T/ipykernel_37820/1702341758.py:55: UserWarning: registration of accessor <class '__main__.register_pandas.<locals>.SeriesAccessor'> under name 'ta' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  class SeriesAccessor(SeriesMethods):


In [14]:
prices.ta.atr(period=14)

date
1980-12-12         NaN
1980-12-15         NaN
1980-12-16         NaN
1980-12-17         NaN
1980-12-18         NaN
                ...   
2024-10-15    4.516121
2024-10-16    4.479971
2024-10-17    4.397830
2024-10-18    4.371556
2024-10-21    4.230731
Length: 11056, dtype: float64

In [15]:
prices.close.ta.sma()

date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18           NaN
                 ...    
2024-10-15    227.524000
2024-10-16    228.078500
2024-10-17    228.242500
2024-10-18    228.582500
2024-10-21    229.082999
Length: 11056, dtype: float64

In [16]:
import inspect
def binding_wrapper(func):
    signature = inspect.signature(func)
    def wrapper(*args, **kwargs):
        bound = signature.bind(*args, **kwargs)
        bound.apply_defaults()
        return bound.arguments
    return wrapper

@binding_wrapper
def myfunc(a=1, b=2):
    return dict(a=a, b=b)

myfunc()

{'a': 1, 'b': 2}